In [25]:
!pip install python-telegram-bot==13.7 firebase-admin==6.2.0

  Attempting uninstall: cachetools
    Found existing installation: cachetools 5.5.1
    Uninstalling cachetools-5.5.1:
      Successfully uninstalled cachetools-5.5.1
  Attempting uninstall: python-telegram-bot
    Found existing installation: python-telegram-bot 21.10
    Uninstalling python-telegram-bot-21.10:
      Successfully uninstalled python-telegram-bot-21.10
  Attempting uninstall: firebase-admin
    Found existing installation: firebase-admin 6.6.0
    Uninstalling firebase-admin-6.6.0:
      Successfully uninstalled firebase-admin-6.6.0


In [3]:
import logging
#import telegram

#from telegram import Update
#from telegram.ext import Application, CommandHandler, ContextTypes
#import firebase_admin
#from firebase_admin import credentials, db

# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Инициализация Firebase
def init_firebase():
    try:
        # Убедитесь, что файл firebase-key.json лежит в той же папке, что и код
        cred = credentials.Certificate("firebase-key.json")
        firebase_admin.initialize_app(cred, {
            'databaseURL': 'https://gpncamp-default-rdb.firebaseio.com/'  # URL из вашего скриншота
        })
        logger.info("Firebase подключен успешно!")
    except Exception as e:
        logger.error(f"Ошибка подключения к Firebase: {e}")
        exit()

# Сохранить пользователя в БД
def save_user(user_id: int, username: str):
    try:
        ref = db.reference(f'/users/{user_id}')
        ref.set({
            'user_id': user_id,
            'username': username,
            'status': 'active'
        })
        logger.info(f"Пользователь {user_id} сохранен в БД")
    except Exception as e:
        logger.error(f"Ошибка сохранения пользователя: {e}")

# Получить данные пользователя из БД
def get_user(user_id: int):
    try:
        ref = db.reference(f'/users/{user_id}')
        return ref.get()
    except Exception as e:
        logger.error(f"Ошибка чтения данных: {e}")
        return None

# Обработчик команды /start
async def start(update: telegram.Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.message.from_user
    save_user(user.id, user.username)
    await update.message.reply_text(
        f"Привет, {user.first_name}! Твой профиль сохранен в базе данных 🔥"
    )

# Обработчик команды /me
async def get_my_data(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user = update.message.from_user
    user_data = get_user(user.id)
    
    if user_data:
        await update.message.reply_text(
            f"📝 Твои данные:\n"
            f"ID: {user_data['user_id']}\n"
            f"Username: @{user_data['username']}\n"
            f"Status: {user_data['status']}"
        )
    else:
        await update.message.reply_text("❌ Твой профиль не найден в базе!")

async def main():
    # Инициализируем Firebase
    init_firebase()

    # Создаем бота
    try:
        application = ApplicationBuilder().token("ВАШ_TELEGRAM_ТОКЕН").build()

        # Регистрируем обработчики команд
        application.add_handler(CommandHandler("start", start))
        application.add_handler(CommandHandler("me", get_my_data))

        # Запускаем бота
        await application.run_polling()
        logger.info("Бот запущен!")
    except Exception as e:
        logger.error(f"Ошибка запуска бота: {e}")

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())

NameError: name 'telegram' is not defined